In [9]:
import stan
import pandas as pd
import nest_asyncio
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

nest_asyncio.apply()

In [10]:
schools_code = """

data {
  int<lower=0> T;   // # time points (equally spaced)
  vector[T] y;      // mean corrected return at time t
  vector[T] X;
  
  vector[T] D_sun;
  vector[T] D_sat;
  vector[T] D_mon;
  vector[T] D_heat;

  vector[T] M;
  //vector[T] L;
}

parameters {
  // test
  real mu;
  real psi;
  real psi_2;
  real psi_3;
  real psi_4;

  real d_sat;
  real d_sun;
  real d_mon;
  real d_heat;

  real beta_1;
  real beta_2;


  vector[T] epsilon_1;
  vector[T] epsilon_2;

  vector<lower=0>[T] xi_d;
  vector<lower=0>[T] xi_u;

  real<lower=0> eta_u;
  real<lower=0> eta_d;

  real kappa_hs;
  real<lower=0> omega_h;
  real<lower=-1,upper=1> phi_h;
  real theta_h;

  array[T] simplex[3] q;

  vector[T] J_s;
  vector[T] h_s;
}
transformed parameters { 
  vector[T] J = J_s;
  vector[T] h = h_s;
  vector[T] y_fit;
  real kappa_h;

  if (kappa_hs > 2) {
    kappa_h = 0;
  }
  else if (kappa_hs < 0) {
    kappa_h = 0;
  }
  else {
    kappa_h = kappa_hs;
  }

  for (t in 1:T) {
    J[t] = -xi_d[t] * q[t][1] + 0 * q[t][2] + xi_u[t] * q[t][3];
  }

  h[1] = 0;
  for (t in 2:T) {
    h[t] = h[t-1] + kappa_h * (theta_h - h[t-1]) + phi_h * epsilon_1[t] + sqrt(omega_h) * epsilon_2[t];
  }

  for (t in 8:800) {
    y_fit[t] = (y[t] - y[t-1]) - beta_1 * (y[t-1] - y[t-2]) - beta_2 * (y[t-2] - y[t-3]) - mu - psi * X[t] - J[t] - d_sat * D_sat[t] - d_sun * D_sun[t] - d_mon * D_mon[t] - d_heat * D_heat[t] - psi_2 * M[t-1];
    // - psi_3 * X[t-1]
    // - psi_2 * M[t-1]
    // - d_heat * D_heat[t]
    // - d_hol * D_hol[t]
  }
}
model {
  // Prior
  beta_1 ~ normal(0, 10);
  beta_2 ~ normal(0, 10);

  mu ~ normal(0, 10);
  psi ~ normal(0, 10);
  psi_2 ~ normal(0, 10);
  psi_3 ~ normal(0, 10);

  d_sat ~ normal(0, 10);
  d_sun ~ normal(0, 10);
  d_mon ~ normal(0, 10);
  d_heat ~ normal(0, 10);

  kappa_hs ~ normal(1, 6);

  omega_h ~ inv_gamma(3, 0.05);
  phi_h ~ normal(0, 0.5 * omega_h);
  theta_h ~ normal(0, 10);

  eta_u ~ inv_gamma(1.86, 0.43);
  eta_d ~ inv_gamma(1.86, 0.43);

  for (t in 1:T) {
    xi_d[t] ~ exponential(eta_d);
    xi_u[t] ~ exponential(eta_u);
  }

  for (t in 1:T) {
    q[t] ~ dirichlet(rep_vector(1, 3));
  }

  for (t in 1:T) {
    epsilon_1[t] ~ normal(0, 1);
    epsilon_2[t] ~ normal(0, 1);
  }

  for (t in 8:800) {
    y_fit[t] ~ normal(0, exp(h[t] / 2));
  }
}
generated quantities {  
  vector[T] y_rep = y;
  for (t in 800:T) {
    y_rep[t] = (y[t-1]) + beta_1 * (y[t-1] - y[t-2]) + beta_2 * (y[t-2] - y[t-3]) + mu + psi * X[t] + J[t] + d_sat * D_sat[t] + d_sun * D_sun[t] + d_mon * D_mon[t] + epsilon_1[t] * sqrt(exp(h[t-1])) + d_heat * D_heat[t] + psi_2 * M[t-1];
    // + psi_3 * X[t-1]
    // + psi_2 * M[t-1]
    // + d_heat * D_heat[t]
    // + d_hol * D_hol[t]
  }
}
"""

In [11]:
def generate_results(zone, hour):
    data = { 
        "T": 1000,
        "y": np.log(pd.read_csv(f'/home/tim/dv/bsv_inference/data_raw/prices_raw/zone_{zone}/zone_{zone}_hour_{hour}.csv')['prices']).to_numpy()[552:],
        "X": np.squeeze(np.log(pd.read_csv(f'/home/tim/dv/bsv_inference/data_raw/temperature_raw/zone_{zone}/zone_{zone}_hour_{hour}.csv', usecols=['temp']).to_numpy()))[552:],

        "M": np.squeeze(np.log(pd.read_csv(f'/home/tim/dv/bsv_inference/data_raw/prices_raw/minimum_prices_{zone}.csv', usecols=['Minimum_Price']).to_numpy()))[552:],
        "D_heat": np.squeeze(pd.read_csv(f'/home/tim/dv/bsv_inference/data/df_heat.csv', usecols=['heating']).astype(int).to_numpy())[552:],

        "D_sun": np.squeeze(pd.read_csv('/home/tim/dv/bsv_inference/data/sundays.csv').to_numpy())[552:],
        "D_sat": np.squeeze(pd.read_csv('/home/tim/dv/bsv_inference/data/saturdays.csv').to_numpy())[552:],
        "D_mon": np.squeeze(pd.read_csv('/home/tim/dv/bsv_inference/data/mondays.csv').to_numpy())[552:]
        }

    posterior = stan.build(schools_code, data=data)
    fit = posterior.sample(num_chains=4)
    df = fit.to_frame()
    df.to_csv(f'~/dv/bsv_inference/results/zone_{zone}_2/zone_{zone}_hour_{hour}_2.csv', index=False)
    return

In [12]:
generate_results(zone=1, hour=1)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_njh0x_fg/model_bheul2q4.stan', line 103, column 25: Argument
    0.05 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_njh0x_fg/model_bheul2q4.stan', line 62, column 7: A
    control flow statement depends on parameter(s): kappa_hs.
Warning in '/tmp/httpstan_njh0x_fg/model_bheul2q4.stan', line 59, column 2: A
    control flow statement depends on parameter(s): kappa_hs.
    calculation.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0

In [54]:
for hour in range(1, 24):
    generate_results(zone=1, hour=hour)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_pm7gtd_r/model_djijhfn2.stan', line 110, column 25: Argument
    0.05 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_pm7gtd_r/model_djijhfn2.stan', line 64, column 7: A
    control flow statement depends on parameter(s): kappa_hs.
Warning in '/tmp/httpstan_pm7gtd_r/model_djijhfn2.stan', line 61, column 2: A
    control flow statement depends on parameter(s): kappa_hs.
    calculation.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_pm7gtd_r/model_djijhfn2.stan', line 110, column 25: Argument
    0.05 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_pm7gtd_r/model_djijhfn2.stan', line 64, column 7: A
    control flow statement depends on parameter(s): kappa_hs.
Warning in '/tmp/httpstan_pm7gtd_r/model_djijhfn2.stan', line 61, column 2: A
    control flow statement depends on parameter(s): kappa_hs.
    calculation.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0

Building...


...


Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_pm7gtd_r/model_djijhfn2.stan', line 110, column 25: Argument
    0.05 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_pm7gtd_r/model_djijhfn2.stan', line 64, column 7: A
    control flow statement depends on parameter(s): kappa_hs.
Warning in '/tmp/httpstan_pm7gtd_r/model_djijhfn2.stan', line 61, column 2: A
    control flow statement depends on parameter(s): kappa_hs.
    calculation.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_pm7gtd_r/model_djijhfn2.stan', line 110, column 25: Argument
    0.05 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_pm7gtd_r/model_djijhfn2.stan', line 64, column 7: A
    control flow statement depends on parameter(s): kappa_hs.
Warning in '/tmp/httpstan_pm7gtd_r/model_djijhfn2.stan', line 61, column 2: A
    control flow statement depends on parameter(s): kappa_hs.
    calculation.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_pm7gtd_r/model_djijhfn2.stan', line 110, column 25: Argument
    0.05 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_pm7gtd_r/model_djijhfn2.stan', line 64, column 7: A
    control flow statement depends on parameter(s): kappa_hs.
Warning in '/tmp/httpstan_pm7gtd_r/model_djijhfn2.stan', line 61, column 2: A
    control flow statement depends on parameter(s): kappa_hs.
    calculation.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0

In [1]:
raw_model = """

data {
  int<lower=0> T;   // # time points (equally spaced)
  vector[T] y;      // mean corrected return at time t
  vector[T] X;
  vector[T] M;
  
  vector[T] D_sun;
  vector[T] D_sat;
  vector[T] D_mon;
  vector[T] D_heat;
}

parameters {
  real mu;
  real psi;
  real psi_2;

  real d_sat;
  real d_sun;
  real d_mon;
  real d_heat;

  vector[T] epsilon_1;
  vector[T] epsilon_2;

  vector<lower=0>[T] xi_d;
  vector<lower=0>[T] xi_u;

  real<lower=0> eta_u;
  real<lower=0> eta_d;

  real kappa_hs;
  real<lower=0> omega_h;
  real<lower=-1,upper=1> phi_h;
  real theta_h;

  array[T] simplex[3] q;

  vector[T] J_s;
  vector[T] h_s;
}
transformed parameters { 
  vector[T] J = J_s;
  vector[T] h = h_s;
  vector[T] y_fit;
  real kappa_h;

  if (kappa_hs > 2) {
    kappa_h = 0;
  }
  else if (kappa_hs < 0) {
    kappa_h = 0;
  }
  else {
    kappa_h = kappa_hs;
  }

  for (t in 1:T) {
    J[t] = -xi_d[t] * q[t][1] + 0 * q[t][2] + xi_u[t] * q[t][3];
  }

  h[1] = 0;
  for (t in 2:T) {
    h[t] = h[t-1] + kappa_h * (theta_h - h[t-1]) + phi_h * epsilon_1[t] + sqrt(omega_h) * epsilon_2[t];
  }

  for (t in 2:800) {
    y_fit[t] = y[t] - y[t-1] - mu - psi * X[t] - J[t] - d_sat * D_sat[t] - d_sun * D_sun[t] - d_mon * D_mon[t];//  - psi_2 * M[t-1] - d_heat * D_heat[t];
    // - psi_3 * X[t-1]
    // - psi_2 * M[t-1]
    // - d_heat * D_heat[t]
    // - d_hol * D_hol[t]
  }
}
model {
  // Prior
  mu ~ normal(0, 10);
  psi ~ normal(0, 10);
  psi_2 ~ normal(0, 10);

  d_sat ~ normal(0, 10);
  d_sun ~ normal(0, 10);
  d_mon ~ normal(0, 10);
  d_heat ~ normal(0, 10);

  kappa_hs ~ normal(1, 6);

  omega_h ~ inv_gamma(3, 0.05);
  phi_h ~ normal(0, 0.5 * omega_h);
  theta_h ~ normal(0, 10);

  eta_u ~ inv_gamma(1.86, 0.43);
  eta_d ~ inv_gamma(1.86, 0.43);

  for (t in 1:T) {
    xi_d[t] ~ exponential(eta_d);
    xi_u[t] ~ exponential(eta_u);
  }

  //alpha = rep_vector(1, 3);
  for (t in 1:T) {
    q[t] ~ dirichlet(rep_vector(1, 3));
  }

  for (t in 1:T) {
    epsilon_1[t] ~ normal(0, 1);
    epsilon_2[t] ~ normal(0, 1);
  }

  for (t in 2:800) {
    y_fit[t] ~ normal(0, exp(h[t] / 2));
  }
}
generated quantities {  
  vector[T] y_rep = y;
  for (t in 800:T) {
    y_rep[t] = y[t-1] + mu + psi * X[t] + J[t] + d_sat * D_sat[t] + d_sun * D_sun[t] + d_mon * D_mon[t] + epsilon_1[t] * sqrt(exp(h[t-1]));// + psi_2 * M[t-1] + d_heat * D_heat[t];
    // + psi_3 * X[t-1]
    // + psi_2 * M[t-1]
    // + d_heat * D_heat[t]
    // + d_hol * D_hol[t]
  }
}
"""

In [2]:
def generate_results_raw(zone, hour):
    data = { 
        "T": 1000,
        "y": np.log(pd.read_csv(f'/home/tim/dv/bsv_inference/data_raw/prices_raw/zone_{zone}/zone_{zone}_hour_{hour}.csv')['prices']).to_numpy()[552:],
        "X": np.squeeze(np.log(pd.read_csv(f'/home/tim/dv/bsv_inference/data_raw/temperature_raw/zone_{zone}/zone_{zone}_hour_{hour}.csv', usecols=['temp']).to_numpy()))[552:],
        "M": np.squeeze(np.log(pd.read_csv(f'/home/tim/dv/bsv_inference/data_raw/prices_raw/minimum_prices_{zone}.csv', usecols=['Minimum_Price']).to_numpy()))[552:],

        "D_sun": np.squeeze(pd.read_csv('/home/tim/dv/bsv_inference/data/sundays.csv').to_numpy())[552:],
        "D_sat": np.squeeze(pd.read_csv('/home/tim/dv/bsv_inference/data/saturdays.csv').to_numpy())[552:],
        "D_mon": np.squeeze(pd.read_csv('/home/tim/dv/bsv_inference/data/mondays.csv').to_numpy())[552:],
        "D_heat": np.squeeze(pd.read_csv(f'/home/tim/dv/bsv_inference/data/df_heat.csv', usecols=['heating']).astype(int).to_numpy())[552:]
        }

    posterior = stan.build(raw_model, data=data)
    fit = posterior.sample(num_chains=4)
    df = fit.to_frame()
    df.to_csv(f'~/dv/bsv_inference/results/zone_{zone}_new_aug/zone_{zone}_hour_{hour}.csv', index=False)
    return

In [14]:
generate_results_raw(zone=2, hour=0)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_lsjgcenc/model_mklxipru.stan', line 94, column 25: Argument
    0.05 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_lsjgcenc/model_mklxipru.stan', line 52, column 7: A
    control flow statement depends on parameter(s): kappa_hs.
Warning in '/tmp/httpstan_lsjgcenc/model_mklxipru.stan', line 49, column 2: A
    control flow statement depends on parameter(s): kappa_hs.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Samplin

In [5]:
for hour in range(24):
    generate_results_raw(zone=2, hour=hour)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_5fxnmtza/model_4r25q5pl.stan', line 90, column 25: Argument
    0.05 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_5fxnmtza/model_4r25q5pl.stan', line 53, column 7: A
    control flow statement depends on parameter(s): kappa_hs.
Warning in '/tmp/httpstan_5fxnmtza/model_4r25q5pl.stan', line 50, column 2: A
    control flow statement depends on parameter(s): kappa_hs.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Samplin

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_5fxnmtza/model_4r25q5pl.stan', line 90, column 25: Argument
    0.05 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_5fxnmtza/model_4r25q5pl.stan', line 53, column 7: A
    control flow statement depends on parameter(s): kappa_hs.
Warning in '/tmp/httpstan_5fxnmtza/model_4r25q5pl.stan', line 50, column 2: A
    control flow statement depends on parameter(s): kappa_hs.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Samplin

In [19]:
augmented_model = """

data {
  int<lower=0> T;   // # time points (equally spaced)
  vector[T] y;      // mean corrected return at time t
  vector[T] X;
  
  vector[T] D_sun;
  vector[T] D_sat;
  vector[T] D_mon;
  vector[T] D_heat;

  vector[T] M;
}

parameters {
  // test
  real mu;
  real psi;
  real psi_2;
  real psi_3;
  real psi_4;

  real d_sat;
  real d_sun;
  real d_mon;
  real d_heat;

  real beta_2;
  real beta_3;

  vector[T] epsilon_1;
  vector[T] epsilon_2;

  vector<lower=0>[T] xi_d;
  vector<lower=0>[T] xi_u;

  real<lower=0> eta_u;
  real<lower=0> eta_d;

  real kappa_hs;
  real<lower=0> omega_h;
  real<lower=-1,upper=1> phi_h;
  real theta_h;

  array[T] simplex[3] q;

  vector[T] J_s;
  vector[T] h_s;
}
transformed parameters { 
  vector[T] J = J_s;
  vector[T] h = h_s;
  vector[T] y_fit;
  real kappa_h;

  if (kappa_hs > 2) {
    kappa_h = 0;
  }
  else if (kappa_hs < 0) {
    kappa_h = 0;
  }
  else {
    kappa_h = kappa_hs;
  }

  for (t in 1:T) {
    J[t] = -xi_d[t] * q[t][1] + 0 * q[t][2] + xi_u[t] * q[t][3];
  }

  h[1] = 0;
  for (t in 2:T) {
    h[t] = h[t-1] + kappa_h * (theta_h - h[t-1]) + phi_h * epsilon_1[t] + sqrt(omega_h) * epsilon_2[t];
  }

  for (t in 8:800) {
    y_fit[t] = y[t] - y[t-1] - beta_1 * y[t-2] - beta_2 * y[t-3] - mu - psi * X[t] - J[t] - d_sat * D_sat[t] - d_sun * D_sun[t] - d_mon * D_mon[t] - d_heat * D_heat[t] - psi_2 * M[t-1];
  }
}
model {
  // Prior
  beta_1 ~ normal(0, 10);
  beta_2 ~ normal(0, 10);

  mu ~ normal(0, 10);
  psi ~ normal(0, 10);
  psi_2 ~ normal(0, 10);
  psi_3 ~ normal(0, 10);

  d_sat ~ normal(0, 10);
  d_sun ~ normal(0, 10);
  d_mon ~ normal(0, 10);
  d_heat ~ normal(0, 10);

  kappa_hs ~ normal(1, 6);

  omega_h ~ inv_gamma(3, 0.05);
  phi_h ~ normal(0, 0.5 * omega_h);
  theta_h ~ normal(0, 10);

  eta_u ~ inv_gamma(1.86, 0.43);
  eta_d ~ inv_gamma(1.86, 0.43);

  for (t in 1:T) {
    xi_d[t] ~ exponential(eta_d);
    xi_u[t] ~ exponential(eta_u);
  }

  for (t in 1:T) {
    q[t] ~ dirichlet(rep_vector(1, 3));
  }

  for (t in 1:T) {
    epsilon_1[t] ~ normal(0, 1);
    epsilon_2[t] ~ normal(0, 1);
  }

  for (t in 8:800) {
    y_fit[t] ~ normal(0, exp(h[t] / 2));
  }
}
generated quantities {  
  vector[T] y_rep = y;
  for (t in 800:T) {
    y_rep[t] = y[t-1] + beta_1 * y[t-2] + beta_2 * y[t-3] + mu + psi * X[t] + J[t] + d_sat * D_sat[t] + d_sun * D_sun[t] + d_mon * D_mon[t] + epsilon_1[t] * sqrt(exp(h[t-1])) + d_heat * D_heat[t] + psi_2 * M[t-1];
  }
}
"""

In [17]:
def generate_results_aug(zone, hour):
    data = { 
        "T": 1000,
        "y": np.log(pd.read_csv(f'/home/tim/dv/bsv_inference/data_raw/prices_raw/zone_{zone}/zone_{zone}_hour_{hour}.csv')['prices']).to_numpy()[552:],
        "X": np.squeeze(np.log(pd.read_csv(f'/home/tim/dv/bsv_inference/data_raw/temperature_raw/zone_{zone}/zone_{zone}_hour_{hour}.csv', usecols=['temp']).to_numpy()))[552:],
        "M": np.squeeze(np.log(pd.read_csv(f'/home/tim/dv/bsv_inference/data_raw/prices_raw/minimum_prices_{zone}.csv', usecols=['Minimum_Price']).to_numpy()))[552:],

        "D_sun": np.squeeze(pd.read_csv('/home/tim/dv/bsv_inference/data/sundays.csv').to_numpy())[552:],
        "D_sat": np.squeeze(pd.read_csv('/home/tim/dv/bsv_inference/data/saturdays.csv').to_numpy())[552:],
        "D_mon": np.squeeze(pd.read_csv('/home/tim/dv/bsv_inference/data/mondays.csv').to_numpy())[552:],
        "D_heat": np.squeeze(pd.read_csv(f'/home/tim/dv/bsv_inference/data/df_heat.csv', usecols=['heating']).astype(int).to_numpy())[552:]
        }

    posterior = stan.build(raw_model, data=data)
    fit = posterior.sample(num_chains=4)
    df = fit.to_frame()
    df.to_csv(f'~/dv/bsv_inference/results/zone_{zone}_new_aug/zone_{zone}_hour_{hour}.csv', index=False)
    return

In [18]:
for hour in range(24):
    generate_results_aug(zone=2, hour=hour)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_5fxnmtza/model_4r25q5pl.stan', line 90, column 25: Argument
    0.05 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_5fxnmtza/model_4r25q5pl.stan', line 53, column 7: A
    control flow statement depends on parameter(s): kappa_hs.
Warning in '/tmp/httpstan_5fxnmtza/model_4r25q5pl.stan', line 50, column 2: A
    control flow statement depends on parameter(s): kappa_hs.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Samplin